In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

import itertools
from importlib import reload
plt=reload(plt)

import warnings
warnings.filterwarnings('ignore')

## IMPORTING THE DATA

In [2]:
df = pd.read_excel("C:/Users/91738/Online Retail Recommendation/input/Online Retail.xlsx")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
df['datetime_col'] = pd.to_datetime(df['InvoiceDate'])
df['date_col']  = df['datetime_col'].dt.date
df['time_col']  = df['datetime_col'].dt.time

dfz = df.copy(deep=True)
dfz = dfz[dfz['Quantity']>=0]

dft = dfz.copy(deep=True)
dft['time_intervals'] = pd.cut(df['datetime_col'].dt.hour, bins=range(0, 25, 4), include_lowest=True, labels=['00:00-03:59', '04:00-07:59', '08:00-11:59', '12:00-15:59', '16:00-19:59', '20:00-23:59'])

dft['year-month_intervals'] = pd.cut(df['datetime_col'].dt.year*100 + df['datetime_col'].dt.month, bins=range(201012, 201201), include_lowest=True) 

labels=['Dec 2010','Jan 2011', 'Feb 2011', 'Mar 2011', 'Apr 2011','May 2011' ,'June 2011' ,'Jul 2011', 'Aug 2011', 'Sep 2011', 'Oct 2011', 'Nov 2011', 'Dec 2011', 'Jan 2012']

ll = LabelEncoder()
dft['year-month_intervals'] = ll.fit_transform(dft['year-month_intervals'])


dff = dft.copy(deep=True)
dff.drop(['datetime_col','time_col','InvoiceDate','date_col'],axis=1,inplace=True)

lk = LabelEncoder()
dff['CustomerID'] = lk.fit_transform(dff['CustomerID'])

In [4]:
L = list(dff["UnitPrice"])
drops = []

for i in range(len(L)):
    if L[i] < 0:
        drops.append(i)

dff = dff.drop(drops)
dff['Total Price'] = dff['Quantity'] * df['UnitPrice']


money_bins = []
A = list(dff["UnitPrice"])

for i in A:
    if 0<=i and i<=5000:
        money_bins.append(0)
    elif 5001<=i and i<=10000:
        money_bins.append(1)
    elif 10001<=i and i<=15000:
        money_bins.append(2)
    elif 15001<=i and i<=20000:
        money_bins.append(3)
    elif 20001<=i and i<=25000:
        money_bins.append(4)
    elif 25001<=i and i<=30000:
        money_bins.append(5)
    elif 30001<=i and i<=35000:
        money_bins.append(6)
    elif 35001<=i and i<=40000:
        money_bins.append(7)
    else:
        money_bins.append(-1)    
dff["Money_bins"] = money_bins

dff.drop("StockCode",axis = 1)

dff.to_csv('dff.csv')

In [ ]:
## Sorting data on the basis of time intervals

df_list_time_intervals=[]
time = list(dff['time_intervals'].unique())
for i in range(len(time)):
  df1 = dff[ dff['time_intervals']==time[i]]
  df1.drop(['time_intervals'],axis=1,inplace=True)
  df_list_time_intervals.append(df1)

In [ ]:
## Sorting data on the basis of year and months

df_list_month_intervals=[]
month = list(dff['year-month_intervals'].unique())
for i in range(len(month)):
  df1 = dff[ dff['year-month_intervals']==month[i]]
  df1.drop(['year-month_intervals'],axis=1,inplace=True)
  df_list_month_intervals.append(df1)


In [ ]:
## Sorting data on the basis of Money

money = list(dff['Money_bins'].unique())
df_list_money_intervals = []

for i in range(len(money)):
    df1 = dff[ dff['Money_bins']==money[i]]
    df1.drop(['Money_bins'],axis=1,inplace=True)
    df_list_money_intervals.append(df1)

In [ ]:
## Sorting data on the basis of Country

country =['United Kingdom', 'Not United Kingdom']
df_list_country_intervals = []

temp_df1 = dff[ dff['Country']=='United Kingdom']
temp_df1.drop(['Country'],axis=1,inplace=True)
df_list_country_intervals.append(temp_df1)
temp_df2 = dff[ dff['Country']!= 'United Kingdom']
temp_df2.drop(['Country'],axis=1,inplace=True)
df_list_country_intervals.append(temp_df2)

In [ ]:
## Creating dictionaries to store the sorted data

time_dict = dict(zip(time,df_list_time_intervals))
month_dict = dict(zip(month,df_list_month_intervals))
money_dict = dict(zip(money,df_list_money_intervals))
country_dict = dict(zip(country,df_list_country_intervals))